In [1]:
import json
import openpyxl
import pandas as pd
import os
import sys
import warnings
from IPython.display import clear_output


sys.path.append("..")
warnings.filterwarnings("ignore", category=FutureWarning)
clear_output()



clear_output()

In [ ]:

# ! pip install openpyxl
# ! pip install pandas
# ! pip install matplotlib
# ! pip install pyperclip

In [2]:
import pandas as pd
from IPython.display import clear_output
import openpyxl
import json
from utils.github_utils import GithubUtils
from utils.java_code_cleaner import JavaCodeCleaner
from utils.code_smells import CodeSmells
from utils.dataset_utils import *

clear_output()

In [3]:
FONTANA_ORIGINAL_DataClass_PATH = "./original_input/fontana_original_data_class.csv"
FONTANA_ORIGINAL_GodClass_PATH = "./original_input/fontana_original_god_class.csv"

FONTANA_ORIGINAL_FeatureEnvy_PATH = "./original_input/fontana_original_feature_envy.csv"
FONTANA_ORIGINAL_LongMethod_PATH = "./original_input/fontana_original_long_method.csv"
FONTANA_ORIGINAL_LongParametersList_PATH = "./original_input/fontana_original_long_parameters_list.csv"
FONTANA_ORIGINAL_SwitchStatement_PATH = "./original_input/fontana_original_switch_statement.csv"

FONTANA_DataClass_PATH = "./input/fontana_data_class.csv"
FONTANA_GodClass_PATH = "./input/fontana_god_class.csv"

FONTANA_FeatureEnvy_PATH = "./input/fontana_feature_envy.csv"
FONTANA_LongMethod_PATH = "./input/fontana_long_method.csv"
FONTANA_LongParametersList_PATH = "./input/fontana_long_parameters_list.csv"
FONTANA_SwitchStatement_PATH = "./input/fontana_switch_statement.csv"

In [4]:
def fix_comma_issue_in_csv_file(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as file:
        raw_text = file.read().replace("'", '"')

    with open(output_path, "w", encoding="utf-8") as file:
        file.write(raw_text)


fix_comma_issue_in_csv_file(
    FONTANA_ORIGINAL_FeatureEnvy_PATH, FONTANA_FeatureEnvy_PATH)

fix_comma_issue_in_csv_file(
    FONTANA_ORIGINAL_LongMethod_PATH, FONTANA_LongMethod_PATH)

In [10]:
import pandas as pd

#! Open & Sort Fontana_Datasets

df1 = pd.read_csv(FONTANA_DataClass_PATH).sort_values(
    by=["IDType", "project", "package", "complextype"])
df2 = pd.read_csv(FONTANA_GodClass_PATH).sort_values(
    by=["IDType", "project", "package", "complextype"])

df3 = pd.read_csv(FONTANA_FeatureEnvy_PATH, on_bad_lines="skip").sort_values(
    by=["IDMethod", "project", "package", "complextype", "method"])
df4 = pd.read_csv(FONTANA_LongMethod_PATH, on_bad_lines="warn").sort_values(
    by=["IDMethod", "project", "package", "complextype", "method"])
df5 = pd.read_csv(FONTANA_LongParametersList_PATH).sort_values(
    by=["IDMethod", "project", "package", "complextype", "method"])
df6 = pd.read_csv(FONTANA_SwitchStatement_PATH).sort_values(
    by=["IDMethod", "project", "package", "complextype", "method"])

allDfs = [df3, df4, df5, df6]

for df in allDfs:
    display(len(df))

/var/folders/2w/crm4crcx47q8y6l_5h_49ldr0000gn/T/ipykernel_6138/1584609847.py:12: ParserWarning: Skipping line 10: expected 88 fields, saw 98
Skipping line 11: expected 88 fields, saw 98

  df4 = pd.read_csv(FONTANA_LongMethod_PATH, on_bad_lines="warn").sort_values(


420

418

420

420

In [11]:
import pandas as pd
github_url_col = "github_url"
github_raw_url_col = "github_raw_url"

for i in range(len(allDfs)):
    # print("Dataset:", allDfs[i].columns[-1][3:].upper())

    allDfs[i] = allDfs[i][list(allDfs[i].columns[:5]) +
                          [allDfs[i].columns[-1]]]

    # print("   Shape:", allDfs[i].shape)
    # print("   Cols:", allDfs[i].columns)


#! Merge all dataframes using outer join on 'IDMethod'
merged_df = pd.concat(allDfs, ignore_index=True).groupby(
    'method', as_index=False).first().fillna(0).astype('object').infer_objects(copy=False).sort_values(
    by=["project", "package", "complextype", "method"]).replace({False: 0, True: 1})

merged_df['project'] = merged_df['project'].str.replace(
    '/mnt/qualitas/qualitas_corpus_compile/', '', regex=False)


merged_df = merged_df.rename(
    columns={'IDMethod': "id"})
merged_df = merged_df.rename(
    columns={'complextype': "class"})
merged_df = merged_df.rename(
    columns={'is_feature_envy': CodeSmells.isFeatureEnvy})
merged_df = merged_df.rename(
    columns={'is_long_method': CodeSmells.isLongMethod})
merged_df = merged_df.rename(
    columns={'is_long_parameters_list': CodeSmells.isLongParametersList})
merged_df = merged_df.rename(
    columns={'is_switch_statements': CodeSmells.isSwitchStatement})

merged_df[github_url_col] = ""
merged_df[github_raw_url_col] = ""


display(merged_df)


smelled_df = merged_df[merged_df.iloc[:, 1:].eq(True).sum(axis=1) >= 1]
display(smelled_df)


#! Save Intermediate Dataset to a CSV file
intermediate_dataset_csv_path = "./output/fontana_dataset.csv"
smelled_df.to_csv(intermediate_dataset_csv_path, index=False)


#! Save Intermediate Dataset to a XLSX file
intermediate_dataset_xlsx_path = "./output/fontana_dataset.xlsx"
smelled_df.to_excel(
    intermediate_dataset_xlsx_path, index=False, engine='openpyxl')

,method,id,project,package,class,isFeatureEnvy,isLongMethod,isLongParametersList,isSwitchStatement,github_url,github_raw_url
1112,transform(Mesh),156976,aoi-2.8.1,artofillusion.animation.distortion,BendDistortion,0,1,0,0,,
45,CustomDistortionTrack(ObjectInfo),156935,aoi-2.8.1,artofillusion.animation.distortion,CustomDistortionTrack,0,0,0,0,,
262,apply(double),156936,aoi-2.8.1,artofillusion.animation.distortion,CustomDistortionTrack,0,0,0,0,,
333,copy(Track),156938,aoi-2.8.1,artofillusion.animation.distortion,CustomDistortionTrack,0,0,0,0,,
420,duplicate(Object),156937,aoi-2.8.1,artofillusion.animation.distortion,CustomDistortionTrack,0,0,0,0,,
...,...,...,...,...,...,...,...,...,...,...,...
813,makeMBeanInfo(),10994,xmojo-5.0.0,com.adventnet.jmx,DefaultDynamicMBean,0,0,0,0,,
1003,setAttribute(Attribute),11010,xmojo-5.0.0,com.adventnet.jmx,DefaultDynamicMBean,1,0,0,0,,
349,"createMBean(String,ObjectName,Object[],String[])",11048,xmojo-5.0.0,com.adventnet.jmx,MBeanServerImpl,0,1,0,0,,
135,"ObjectInputStreamSupport(InputStream,ClassLoader)",10990,xmojo-5.0.0,com.adventnet.jmx,ObjectInputStreamSupport,0,0,0,0,,


,method,id,project,package,class,isFeatureEnvy,isLongMethod,isLongParametersList,isSwitchStatement,github_url,github_raw_url
1112,transform(Mesh),156976,aoi-2.8.1,artofillusion.animation.distortion,BendDistortion,0,1,0,0,,
753,"interpOneAxis(MeshVertex[][],float[],double[][...",158055,aoi-2.8.1,artofillusion.object,SplineMesh,0,1,0,0,,
389,"doSubdivide(TriangleMesh,Vertex[],Edge[],Face[...",157307,aoi-2.8.1,artofillusion.object,TriangleMesh,0,1,0,0,,
344,"createFinalImage(Vec3,Vec3,Vec3,Vec3)",162348,aoi-2.8.1,artofillusion.raster,Raster,1,1,0,0,,
953,"renderTriangleGouraud(Vec2,float,double,double...",162358,aoi-2.8.1,artofillusion.raster,Raster,1,1,0,0,,
...,...,...,...,...,...,...,...,...,...,...,...
503,"fixupCurrentEntity(XMLEntityManager,char[],int)",179846,xerces-2.10.0,org.apache.xerces.impl,XMLVersionDetector,1,0,0,0,,
1082,"startDocumentParsing(XMLEntityHandler,short)",179844,xerces-2.10.0,org.apache.xerces.impl,XMLVersionDetector,1,0,0,0,,
755,"invoke(String,Object[],String[])",11003,xmojo-5.0.0,com.adventnet.jmx,DefaultDynamicMBean,1,1,0,0,,
1003,setAttribute(Attribute),11010,xmojo-5.0.0,com.adventnet.jmx,DefaultDynamicMBean,1,0,0,0,,


In [ ]:
import pandas as pd
import json

intermediate_dataset_path = "./output/fontana_dataset.csv"
mld_json_file_path = "./output/fontana_mld_sc.json"


#! Open Intermediate Dataset
intermediateDf = pd.read_csv(
    intermediate_dataset_path, on_bad_lines="skip")


mld_items = []
for i, row in intermediateDf.iterrows():
    source_code = ""
    if not source_code:
        processed_source_code = JavaCodeCleaner.clean(source_code)
        code_smells = []
        if row[CodeSmells.isFeatureEnvy] == 1:
            code_smells.append(CodeSmells.FeatureEnvy)
        if row[CodeSmells.isLongMethod] == 1:
            code_smells.append(CodeSmells.LongMethod)
        if row[CodeSmells.isLongParametersList] == 1:
            code_smells.append(CodeSmells.LongParametersList)
        if row[CodeSmells.isSwitchStatement] == 1:
            # if row["class"] == "Attribute_LocalVariableTable":
            #     display(row)
            code_smells.append(CodeSmells.SwitchStatement)

        item = {
            "code_smells": code_smells,
            "project": row["project"],
            "package": row["package"],
            "class": row["class"],
            "method": row["method"],
            "github_url": "",
            "source_code": ""
        }
        mld_items.append(item)


with open(mld_json_file_path, 'w') as mld_json:
    json.dump(mld_items, mld_json)
    mld_json.write('\n')

In [3]:
import json

mld_json_file_path_413 = "./output/fontana_mld_sc_powered.json"
intermediate_dataset_path = "./output/fontana_dataset.csv"

intermediateDf = pd.read_csv(
    intermediate_dataset_path, on_bad_lines="skip")

# display(intermediateDf)

with open(mld_json_file_path_413, 'r') as json_file:
    json_dataset = json.load(json_file)

display(len(json_dataset))


for item in json_dataset:
    if item["github_url"] != "":
        intermediateDf.loc[
            (intermediateDf["package"] == item["package"]) &
            (intermediateDf["method"] == item["method"]) &
            (intermediateDf["class"] == item["class"]),
            "github_url"
        ] = item["github_url"]

        intermediateDf.loc[
            (intermediateDf["package"] == item["package"]) &
            (intermediateDf["method"] == item["method"]) &
            (intermediateDf["class"] == item["class"]),
            "github_raw_url"
        ] = GithubUtils.convert_blob_to_raw_url(item["github_url"])

#! Remove Rows Have No Github URL
intermediateDf = intermediateDf[
    (intermediateDf["github_url"] != "") &
    (intermediateDf["github_url"].notna())
]

display(intermediateDf)

# #! Save Intermediate Dataset to a CSV file
final_dataset_csv_path = "./output/fontana_final_dataset.csv"
intermediateDf.to_csv(final_dataset_csv_path, index=False)


# #! Save Intermediate Dataset to a XLSX file
# final_dataset_xlsx_path = "./output/fontana_final_dataset.xlsx"
# intermediateDf.to_excel(
#     final_dataset_xlsx_path, index=False, engine='openpyxl')

412

,method,id,project,package,class,isFeatureEnvy,isLongMethod,isLongParametersList,isSwitchStatement,github_url,github_raw_url
0,transform(Mesh),156976,aoi-2.8.1,artofillusion.animation.distortion,BendDistortion,0,1,0,0,https://github.com/JavaQualitasCorpus/aoi-2.8....,https://raw.githubusercontent.com/JavaQualitas...
1,"interpOneAxis(MeshVertex[][],float[],double[][...",158055,aoi-2.8.1,artofillusion.object,SplineMesh,0,1,0,0,https://github.com/JavaQualitasCorpus/aoi-2.8....,https://raw.githubusercontent.com/JavaQualitas...
2,"doSubdivide(TriangleMesh,Vertex[],Edge[],Face[...",157307,aoi-2.8.1,artofillusion.object,TriangleMesh,0,1,0,0,https://github.com/JavaQualitasCorpus/aoi-2.8....,https://raw.githubusercontent.com/JavaQualitas...
3,"createFinalImage(Vec3,Vec3,Vec3,Vec3)",162348,aoi-2.8.1,artofillusion.raster,Raster,1,1,0,0,https://github.com/JavaQualitasCorpus/aoi-2.8....,https://raw.githubusercontent.com/JavaQualitas...
4,"renderTriangleGouraud(Vec2,float,double,double...",162358,aoi-2.8.1,artofillusion.raster,Raster,1,1,0,0,https://github.com/JavaQualitasCorpus/aoi-2.8....,https://raw.githubusercontent.com/JavaQualitas...
...,...,...,...,...,...,...,...,...,...,...,...
407,"executePredicates(int, XPathContext)",310,xalan-2.7.1,org.apache.xpath.axes,PredicatedNodeTest,0,0,0,1,https://github.com/JavaQualitasCorpus/xalan-2....,https://raw.githubusercontent.com/JavaQualitas...
408,"findProviderClass(String, ClassLoader, boolean)",554,xalan-2.7.1,org.apache.xpath.functions,ObjectFactory,0,0,0,1,https://github.com/JavaQualitasCorpus/xalan-2....,https://raw.githubusercontent.com/JavaQualitas...
409,"setupCurrentEntity(String,XMLInputSource,boole...",179871,xerces-2.10.0,org.apache.xerces.impl,XMLEntityManager,1,1,0,0,https://github.com/JavaQualitasCorpus/xerces-2...,https://raw.githubusercontent.com/JavaQualitas...
410,"fixupCurrentEntity(XMLEntityManager,char[],int)",179846,xerces-2.10.0,org.apache.xerces.impl,XMLVersionDetector,1,0,0,0,https://github.com/JavaQualitasCorpus/xerces-2...,https://raw.githubusercontent.com/JavaQualitas...


In [4]:
import json
import requests
import re
import matplotlib.pyplot as plt
final_dataset_path = "./output/fontana_final_dataset.csv"


#! Open Intermediate Dataset
finalDf = pd.read_csv(
    final_dataset_path, on_bad_lines="skip")

display(finalDf)


mld_items = []
for i, row in finalDf.iterrows():
        if i % 10 == 0:
            clear_output()
        display(i)
        source_code = GithubUtils.clone_code(row["github_raw_url"])
        if source_code:
            source_code = "class C {\n" + source_code + "\n}"
            processed_source_code = JavaCodeCleaner.clean_by_cleaner_api(
                source_code
            )
            processed_source_code = processed_source_code[8:-1]
            if processed_source_code:
                code_smells = []
                if row[CodeSmells.isFeatureEnvy] == 1:
                    code_smells.append(CodeSmells.FeatureEnvy)
                if row[CodeSmells.isLongMethod] == 1:
                    code_smells.append(CodeSmells.LongMethod)
                if row[CodeSmells.isLongParametersList] == 1:
                    code_smells.append(CodeSmells.LongParametersList)
                if row[CodeSmells.isSwitchStatement] == 1:
                    code_smells.append(CodeSmells.SwitchStatement)

                item = JsonDatasetUtils.create_item(
                    code_smells, processed_source_code
                )
                mld_items.append(item)

390

https://raw.githubusercontent.com/JavaQualitasCorpus/xalan-2.7.1/13cf15013547bd021da175c2289737cf57b31c4c/src/org/apache/xml/utils/DOMHelper.java#L340-L459


391

https://raw.githubusercontent.com/JavaQualitasCorpus/xalan-2.7.1/13cf15013547bd021da175c2289737cf57b31c4c/src/org/apache/xpath/axes/DescendantIterator.java#L44-L126


392

...

https://raw.githubusercontent.com/JavaQualitasCorpus/xerces-2.10.0/a00a5986b24d3b762ba230e8d44a20fae6ecd58a/src/org/apache/xerces/impl/XMLEntityManager.java#L927-L1190


395

https://raw.githubusercontent.com/JavaQualitasCorpus/xerces-2.10.0/a00a5986b24d3b762ba230e8d44a20fae6ecd58a/src/org/apache/xerces/impl/XMLVersionDetector.java#L213-L239


396

https://raw.githubusercontent.com/JavaQualitasCorpus/xerces-2.10.0/a00a5986b24d3b762ba230e8d44a20fae6ecd58a/src/org/apache/xerces/impl/XMLVersionDetector.java#L108-L131


In [5]:
display(len(mld_items))
display(mld_items)

397

[{'code_smells': ['LongMethod'],
  'source_code': 'public Mesh transform(Mesh obj){if(previous!=null)obj=previous.transform(obj);Mesh newmesh=(Mesh)obj.duplicate();MeshVertex[]vert=newmesh.getVertices();Vec3[]newvert=new Vec3[vert.length];Vec3 origin=new Vec3();for(int i=0;i<newvert.length;i++){newvert[i]=vert[i].r;if(preTransform!=null)preTransform.transform(newvert[i]);}if(preTransform!=null)preTransform.transform(origin);double min=Double.MAX_VALUE,max=Double.MIN_VALUE;for(int i=0;i<newvert.length;i++){double value;if(axis==X_AXIS)value=newvert[i].x;else if(axis==Y_AXIS)value=newvert[i].y;else value=newvert[i].z;if(value<min)min=value;if(value>max)max=value;}if(min>=max)return obj;if(!forward){double temp=min;min=max;max=temp;}double theta=angle*(Math.PI/180.0);double scale=theta/(max-min);double radius=(max-min)/theta;Vec3 center;if(axis==X_AXIS){if(direction==Y_AXIS)for(int i=0;i<newvert.length;i++){double a=scale*(newvert[i].x-min);double b=newvert[i].y-origin.y-radius;newvert[i]

In [ ]:
import pyperclip

# code = """

# \n\tpublic boolean execute (MessageState msgState, String param) {\r\n\t\tif (isPunished (msgState)) \r\n\t\t\treturn false;\r\n\t\t// check if sender does have the right to kick a user\r\n\t\tString grpname = null;\r\n//\t\tGroup sg = msgState.sender.getGroup ();\r\n\t\tboolean mayKick = msgState.sender.hasRight(IUserStates.MAY_KICK);\r\n\t\tif (!mayKick) {\r\n            msgState.msgTemplate = "error.noRight.noVipAdmin";\r\n            msgState.sender.sendMessage(msgState.mp);\r\n            return false;\r\n        }\r\n        int pos1 = param.indexOf(" ");\r\n        int pos2 = param.indexOf(":");\r\n        boolean multiUser = (pos1!=-1 && (pos2 > pos1+1 || pos2==-1));\r\n        if (multiUser \r\n        \t\t&& !msgState.sender.hasRight(IUserStates.ROLE_VIP)) {\r\n            msgState.msgTemplate="error.mass.noRight.noVipAdmin";\r\n            msgState.sender.sendMessage (msgState.mp);\r\n            return false;\r\n        }\r\n\t\tif (pos2>-1) {\r\n            String parts[] = param.split (":");\r\n\t\t\tparam = parts[0];\r\n\t\t\tmsgState.targetGroup = GroupManager.mgr.getGroup(parts[1]);\r\n\t\t\tgrpname = parts[1];\r\n\t\t} else {\r\n        \tmsgState.msgTemplate = "error.kc.noroom";\r\n            msgState.sender.sendMessage(msgState.mp);\r\n        \treturn false;\r\n        }\r\n        Vector<User> found = getMultibleUsers(msgState, param);\r\n        boolean created = false;\r\n        GroupUserList gul = new GroupUserList();\r\n        msgState.useRenderCache=false;\r\n        for (Iterator<User> i = found.iterator(); i.hasNext(); ) {\r\n            User cu = (User) i.next();\r\n            msgState.sourceGroup = cu.getGroup();\r\n            if (cu.hasRight(IUserStates.ROLE_GOD)) {\r\n            \tmsgState.usercontext = cu;\r\n                msgState.msgTemplate = "error.noRight.isAdmin";\r\n                msgState.sender.sendMessage(msgState.mp);\r\n                i.remove();\r\n                msgState.param = grpname;\r\n                msgState.msgTemplate = "message.kc.godinfo";\r\n                cu.sendMessage(msgState.mp);\r\n                continue;\r\n            } else if (cu.hasRight(IUserStates.ROLE_VIP)\r\n                       && !msgState.sender.hasRight(IUserStates.ROLE_GOD)) {\r\n            \tmsgState.usercontext = cu;               \r\n            \tmsgState.msgTemplate = "error.noRight.isVip";\r\n                msgState.sender.sendMessage(msgState.mp);\r\n                i.remove();\r\n                continue;\r\n            } else if (cu.hasRight(IUserStates.PROTECTED_FROM_KICK)\r\n                       && !msgState.sender.hasRight(IUserStates.ROLE_GOD)) {\r\n                msgState.msgTemplate = "error.k.protected";\r\n                msgState.usercontext = cu;\r\n                msgState.sender.sendMessage(msgState.mp);\r\n                i.remove();\r\n                continue;\r\n            } else if (!msgState.sender.hasRight(IUserStates.ROLE_GOD)\r\n                    && !isVip (msgState.sender, cu.getGroup(), msgState)) {\r\n\t\t\t    msgState.targetGroup = cu.getGroup();\r\n\t\t\t    if (msgState.usrList.length > 1)\r\n\t\t\t        msgState.msgTemplate = "error.membership.notMember.many";\r\n\t\t\t    else\r\n\t\t\t        msgState.msgTemplate = "error.membership.notMember";\r\n                msgState.usercontext=cu;\r\n                msgState.sender.sendMessage(msgState.mp);\r\n                i.remove();\r\n                continue;\r\n            }\r\n            if (msgState.targetGroup == null) {\r\n                msgState.targetGroup = GroupManager.mgr.openGroup (grpname, "punished users here", cu, msgState.sender);\r\n                if (msgState.targetGroup == null) {\r\n                    msgState.param = "";\r\n                    msgState.usercontext = cu;\r\n                    msgState.msgTemplate = "error.k.noGroupRight";\r\n                    msgState.sender.sendMessage(msgState.mp);\r\n                    return false;\r\n                }\r\n                created=true;              \r\n            } else if (!msgState.targetGroup.usrMayJoin(msgState.sender)) {\r\n                msgState.param = "";\r\n                msgState.usercontext = cu;\r\n                msgState.msgTemplate = "error.k.noGroupRight";\r\n                msgState.sender.sendMessage (msgState.mp);\r\n                return false;\r\n            }\r\n            msgState.usercontext = cu;\r\n            gul.addUser(cu);\r\n            msgState.targetGroup.addUser(cu, msgState.sender);\r\n            msgState.targetGroup.removeFromSusers(cu);\r\n            msgState.msgTemplate = "message.kc.personal";\r\n            cu.sendMessage(msgState.mp);\r\n        }\r\n\t\tif (found.size()==0)\r\n            return false;\r\n\t\tif (found.size()==1) {\r\n            msgState.msgTemplate="message.kc.destination.singular";\r\n            msgState.targetGroup.exclusiveSendMessage(msgState.mp, found);\r\n            if (!msgState.sender.getGroup().equals(msgState.sourceGroup)) {\r\n                msgState.msgTemplate="message.kc.confirm.singular";\r\n                msgState.sender.sendMessage(msgState.mp);\r\n            }\r\n            if (msgState.sourceGroup != null) {\r\n                msgState.msgTemplate="message.kc.singular";\r\n                msgState.sourceGroup.sendMessage(msgState.mp);\r\n            }\r\n            return true;\r\n        }\r\n        msgState.usrList = found.toArray();\r\n        gul.sendMessage(msgState, "message.kc", false);\r\n\t\tmsgState.msgTemplate="message.kc.destination.plural";\r\n        msgState.targetGroup.exclusiveSendMessage(msgState.mp, found);\r\n        msgState.msgTemplate="message.kc.confirm.plural";\r\n        msgState.sender.sendMessage(msgState.mp);\r\n\t\treturn true;\r\n\t}\r\n}\r\n

# """

# code = code.replace("\r\n", "\n").replace("\r", "\n")
# display(code)
code = """
    /**
     * Sets the input source for the document to parse.
     *
     * @param inputSource The document's input source.
     *
     * @exception XMLConfigurationException Thrown if there is a 
     *                        configuration error when initializing the
     *                        parser.
     * @exception IOException Thrown on I/O error.
     *
     * @see #parse(boolean)
     */
    public void setInputSource(XMLInputSource inputSource)
        throws XMLConfigurationException, IOException {
        reset();
        fCloseStream = inputSource.getByteStream() == null &&
                       inputSource.getCharacterStream() == null;
        fDocumentScanner.setInputSource(inputSource);
    } // setInputSource(XMLInputSource)
"""

code = "class C {" + code + "}"

code = JavaCodeCleaner.clean_single_line_comments(code)
code = JavaCodeCleaner.clean_extra_space(code)

# display(code)
code = JavaCodeCleaner.clean_by_cleaner_api(code)

new_code = code[8:-1].replace("\"", "'")

display(new_code)


pyperclip.copy(new_code)

In [ ]:
unique_values = smelled_df['project'].unique()

# Print the unique values
print(unique_values)

In [ ]:
import requests


import requests


def get_github_repos(username):
    url = f'https://api.github.com/users/{username}/repos'
    repos = []
    page = 1
    while True:
        # Fetch 100 repos per page
        response = requests.get(url, params={'page': page, 'per_page': 116})
        if response.status_code == 200:
            data = response.json()
            if not data:  # If there are no more repos, break the loop
                break
            repos.extend([repo['name'] for repo in data])
            page += 1  # Move to the next page
        else:
            print(f"Error: {response.status_code}")
            break
    return repos


# Example usage:
username = 'JavaQualitasCorpus'  # Replace with the GitHub username you want
repos = get_github_repos(username)
repos.sort()
print(len(repos))

In [ ]:
unique_values.sort()
print(unique_values)

In [ ]:


# Finding common items using set intersection
common_items = list(set(unique_values) & set(repos))
common_items.sort()

# Print the common items
print(common_items)
print(len(unique_values))

In [6]:
import json

mld_json_file_path = "./output/fontana_mld_sc.json"

with open(mld_json_file_path, 'w') as mld_json:
    json.dump(mld_items, mld_json)
    mld_json.write('\n')